In [1]:
import requests

In [2]:
#PART 1
def fetch_probe_list():
  #Function to return list of active Starlink Probes
  starlink_probe_list = []
  api_endpoint = "https://atlas.ripe.net/api/v2/probes/?asn_v4=14593&status=1"

  params = {
      "asn_v4": 14593,  #Starlink ASN number
      "status": 1       #Connected Probes
  }

  response = requests.get(api_endpoint, params=params)
  json_data = response.json()

  for probe in json_data["results"]:
    starlink_probe_list.append(probe["id"])

  print("Number of Probes: ", len(starlink_probe_list))
  return starlink_probe_list

In [3]:
fetch_probe_list()

Number of Probes:  56


[28,
 10743,
 10920,
 13040,
 17979,
 19983,
 20544,
 22802,
 23127,
 24742,
 26696,
 26834,
 28221,
 35042,
 35681,
 35751,
 50017,
 52422,
 52918,
 52955,
 53798,
 54330,
 55492,
 60323,
 60510,
 60929,
 61081,
 61113,
 61241,
 61366,
 61532,
 61537,
 61731,
 61780,
 61878,
 61899,
 62083,
 62365,
 62417,
 62498,
 62553,
 62613,
 62741,
 62843,
 62868,
 63017,
 1001747,
 1002750,
 1003131,
 1004232,
 1004453,
 1004876,
 1004978,
 1005561,
 1005623,
 1005627]

## Credit Calculation
It costs 1 credits per ping measurement (1 packets of 48 octets). We have 56 active probes connected to Starlink. If we measure every minutes for an entire day, that is 4*24 = 96 measurements. 1 * 56 * 96 = 5376 credits

In [12]:
definitions = {
    "definitions": [
      {
        "type": "ping",
        "target": "google.com",
        "af": 4,
        "description": "pingtest",
        "is_oneoff": "true",
      }
    ],
    "probes" : [
      {"requested": 1,
       "type": "probes",
       "value": "10920"
      }
    ]
}

def create_measurement(defs):
    api_endpoint = "https://atlas.ripe.net/api/v2/measurements"
    params={"key": '1721b7b6-100f-48b0-ad03-e2b6c95fb978'}

    x = requests.post(api_endpoint, json = defs, params=params)
    json_data = x.json()
    return json_data



In [13]:
create_measurement(definitions)

{'measurements': [52727092]}

In [14]:
def get_results(id):
    api_endpoint = "https://atlas.ripe.net/api/v2/measurements/" + id + "/results"
    x = requests.get(api_endpoint)
    json_data = x.json()
    return json_data

In [15]:
get_results(1)

[{'fw': 5080,
  'mver': '2.6.2',
  'lts': 6,
  'dst_name': '142.250.179.206',
  'af': 4,
  'dst_addr': '142.250.179.206',
  'src_addr': '192.168.10.88',
  'proto': 'ICMP',
  'ttl': 113,
  'size': 64,
  'result': [{'rtt': 34.045007}, {'rtt': 40.057548}, {'rtt': 41.63382}],
  'dup': 0,
  'rcvd': 3,
  'sent': 3,
  'min': 34.045007,
  'max': 41.63382,
  'avg': 38.5787916667,
  'msm_id': 52727092,
  'prb_id': 10920,
  'timestamp': 1682631567,
  'msm_name': 'Ping',
  'from': '145.224.73.88',
  'type': 'ping',
  'group_id': 52727092,
  'step': None,
  'stored_timestamp': 1682631567},
 {'fw': 5080,
  'mver': '2.6.2',
  'lts': 9,
  'dst_name': '142.250.179.206',
  'af': 4,
  'dst_addr': '142.250.179.206',
  'src_addr': '192.168.1.110',
  'proto': 'ICMP',
  'ttl': 58,
  'size': 64,
  'result': [{'rtt': 52.819345}, {'rtt': 44.352708}, {'rtt': 43.667051}],
  'dup': 0,
  'rcvd': 3,
  'sent': 3,
  'min': 43.667051,
  'max': 52.819345,
  'avg': 46.946368,
  'msm_id': 52727092,
  'prb_id': 13040,
  't